In [ ]:
import numpy as np
from numpy import ndarray
import os
import pyperclip
import glob
import sklearn
from sklearn import preprocessing
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt 
import cv2
HOME = str(Path.home() / "Downloads")
print(HOME)

In [ ]:
image_dict = {}
image_final_anns = {}
for _image_ in glob.glob(f'{HOME}/Trichromes/5GY 20W PR8/743/*'): #accesses folder of trichromes
    print(os.path.basename(_image_))
    with Image.open(_image_) as image:
        image = image.convert('RGB')
        image_arr = np.array(image)
        #images.append(image_arr) #puts processed images into list images[]
        image_dict[f'{os.path.basename(_image_)}'] = image_arr

In [ ]:
def count_blue(images):
    percentages = {}
    for key, image_arr in images.items():

        threshold = ~(threshold_image_arr(image_arr)) #where image is not white
        only_blue = ((image_arr[:,:,2]>image_arr[:,:,1]) & (image_arr[:,:,2]>(image_arr[:,:,0]*2.5))) #where image is definitively blue
        gray = (((abs(image_arr[:,:,0]-image_arr[:,:,1]))<=30) & ((abs(image_arr[:,:,1]-image_arr[:,:,2]))<=30) & ((abs(image_arr[:,:,0]-image_arr[:,:,2]))<=30)) #where image is gray
        dark = ((image_arr[:,:,0]+image_arr[:,:,1]+image_arr[:,:,2])<=110) #where image is dark
        
        mask =  (threshold & only_blue & (~gray) & (~dark))

        mask = ~mask #to show black-on-white

        plt.imshow(mask, cmap='gray')
        plt.axis('off')
        plt.style.use("dark_background")
        plt.show()

        image_final_anns[key] = mask

        mask = ~mask

        bluepixels = np.sum(mask)
        size = (np.sum(threshold))
        percentages[key] = (round((bluepixels/size)*100, 5))

    return percentages

def threshold_image_arr(image_arr):
    img_gray = cv2.cvtColor(image_arr, cv2.COLOR_RGB2GRAY)

    histogram_original,_ = np.histogram(img_gray.flatten(), bins=256, range=(0,255))
    histogram = histogram_original[1:]

    reshaped_histogram = histogram.reshape(-1, 1)

    scaler = preprocessing.MinMaxScaler()
    normalized_histogram = scaler.fit_transform(reshaped_histogram)
    normalized_histogram = normalized_histogram.flatten()

    first_derivative = np.gradient(normalized_histogram)
    second_derivative = np.gradient(first_derivative)

    cutoff = 0.0005

    region_of_interest = second_derivative[np.argmax(second_derivative[150:])-30:np.argmax(second_derivative[150:])]
    hill_start_index = np.argmax(second_derivative[150:])-30 + np.argmax(region_of_interest > cutoff)

    threshmask = cv2.threshold(img_gray, hill_start_index, 255, cv2.THRESH_BINARY)[1]

    return threshmask.astype(bool)

string=""
for key, p in count_blue(image_dict).items():
    print(p, " %")
    string += f'{key}\t{p}\n'

pyperclip.copy(string)

for key, value in image_final_anns.items():
    im = Image.fromarray(value)
    im.save(f'{HOME}/Trichrome Image Masks/5GY 20W PR8/743/{key}')